### 自定义要搜索的关键词

In [1]:
# 要搜索的字段
text = '皮带'

# 一共要爬取的数量
totalnum = 10


# 【这一步需要提前手动操作】
# 运行cmd命令
# 输入后面的代码，把【chrome.exe】替换成你的谷歌浏览器的地址 #  chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\selenum\AutomationProfile"
# 就像这样 # "C:\Users\lenovo\Desktop\chrome-win64\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\selenum\AutomationProfile"
#"C:\Users\lenovo\Desktop\chrome-win64\chrome.exe"

# 一、实例化selenium

In [6]:
# 在这里下载合适的版本 
# https://chromedriver.chromium.org/home
# https://googlechromelabs.github.io/chrome-for-testing/


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from selenium.webdriver.chrome.service import Service

import time
from scrapy.selector import Selector
import json
import os


options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
options.add_argument('--incognito')  # 隐身模式（无痕模式）
options.add_argument('blink-settings=imagesEnabled=false')  # 不加载图片, 提升速度
browser = webdriver.Chrome(options=options)
# 设置 ChromeDriver 的路径
service = Service("C:/Users/lenovo/Desktop/xiaohongshu/chromedriver.exe")  # 替换为你的实际路径
browser = webdriver.Chrome(service=service, options=options)

#"C:\Users\lenovo\Desktop\xiaohongshu\chromedriver.exe"

In [7]:
# 要搜索的字段
text = text

# 一共要爬取的数量
totalnum = totalnum

# 打开网页
browser.get('https://www.xiaohongshu.com/explore')

# 判断是否登录
while True:
    page_source = browser.page_source
    if '登录探索更多内容' in page_source:
        time.sleep(2)
        print('%s : 未登录，请手动登录小红书！！！' % time.ctime())
        continue
    else:
        print('%s : 登录成功！！！' % time.ctime())
        time.sleep(3)
        break

# 打开网址
url = r'https://www.xiaohongshu.com/search_result?keyword=%s&source=web_explore_feed' % text
browser.get(url)
time.sleep(2)

# 等待加载
while True:
    if text in browser.title:
        print('%s : 加载成功！！！' % time.ctime())
        break
    else:
        time.sleep(1)

Sun Nov 24 12:00:56 2024 : 登录成功！！！
Sun Nov 24 12:01:08 2024 : 加载成功！！！


In [1]:
import os
import json
from tqdm import tqdm
from scrapy.selector import Selector
import time

# 先加载已经存储的
if os.path.exists('_items.txt'):
    with open('_items.txt', 'r', encoding='utf-8') as f:
        items = f.readlines()
    items = [json.loads(item.strip()) for item in items if item.strip()]  # 确保每行都有内容
    itemCodeList = [item['code'] for item in items]
else:
    itemCodeList = []

# 定义进度条
totalnum = totalnum - len(itemCodeList)
qbar = tqdm(total=totalnum)

# 解析网页
def parsePage(text):
    selector = Selector(text=text)
    # 使用更宽松的 XPath 匹配
    divs = selector.xpath('//section[contains(@class, "note-item")]')
    if not divs:
        print("No items found on page.")
        return

    for div in divs:
        item = {}
        try:
            # 提取数据
            item['title'] = div.xpath('.//a[@class="title"]/span/text()').extract_first()
            item['user'] = div.xpath('.//a[@class="author"]//span[@class="name"]/text()').extract_first()
            like_num = div.xpath('.//span[@class="count"]/text()').extract_first()
            item['likeNum'] = like_num.strip() if like_num else "0"
            href = div.xpath('.//a[@class="cover mask"]/@href').extract_first()
            if not href:
                href = div.xpath('.//a/@href').extract_first()  # 备用路径
            if not href:
                raise ValueError("Missing href")
            item['code'] = href.split('/')[-1]

            # 保存数据
            if item['code'] not in itemCodeList:
                itemCodeList.append(item['code'])
                with open('_items.txt', 'a+', encoding='utf-8') as f:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')
                qbar.update(1)
        except Exception as e:
            print(f"Error parsing item: {e}")


# 爬取数据
while len(itemCodeList) < totalnum:
    if '- THE END -' in browser.page_source:
        print('- THE END -')
        break

    parsePage(text=browser.page_source)
    time.sleep(0.3)
    # 滚到底部
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(2)

qbar.close()
print("Crawling completed!")


NameError: name 'totalnum' is not defined

# 三、抓取详情页

In [9]:
pip install selenium-stealth


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


没有反爬虫肯定可以爬

In [10]:
# 检查已有的 items 数据
if os.path.exists('_items.txt'):
    with open('_items.txt', 'r', encoding='utf-8') as f:
        items = f.readlines()
    items = [json.loads(item.strip()) for item in items]
    itemCodeList = [item['code'] for item in items]
else:
    itemCodeList = []

# 检查已有的详情数据
if os.path.exists('_items_detail.txt'):
    with open('_items_detail.txt', 'r', encoding='utf-8') as f:
        items_detail = f.readlines()
    items_detail = [json.loads(item.strip()) for item in items_detail]
    itemCodeList_detail = [item['code'] for item in items_detail]
else:
    itemCodeList_detail = []

# 开始解析
qbar = tqdm(total=len(items))
for item in items:
    code = item['code']
    if code in itemCodeList_detail:
        continue
    
    # 构建目标URL
    test_url = f'https://www.xiaohongshu.com/discovery/item/{code}'
    browser.get(test_url)

    # 等待页面加载完成
    try:
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@class="list-container"]'))
        )
    except Exception as e:
        print(f"Failed to load page for code {code}: {e}")
        continue

    # 获取页面源代码
    html = browser.page_source
    selector = Selector(text=html)

    try:
        # 提取详情信息
        item['title2'] = selector.xpath('//*[@name="og:title"]/@content').extract_first()
        item['description'] = selector.xpath('//*[@name="description"]/@content').extract_first()
        item['keywords'] = selector.xpath('//*[@name="keywords"]/@content').extract_first()
        item['type'] = selector.xpath('//*[@name="og:type"]/@content').extract_first()
        item['image'] = selector.xpath('//*[@name="og:image"]/@content').extract_first()
        item['url'] = selector.xpath('//*[@name="og:url"]/@content').extract_first()

        # 提取评论信息
        comments_data = []
        comments = selector.xpath('//div[@class="parent-comment"]')
        for comment in comments:
            try:
                user_name = comment.xpath('.//div[@class="author"]/a[@class="name"]/text()').get()
                comment_text = comment.xpath('.//div[@class="content"]//span[@class="note-text"]/span/text()').getall()
                comment_text = ''.join(comment_text)  # 拼接评论文本
                location = comment.xpath('.//div[@class="info"]//span[@class="location"]/text()').get()
                
                comments_data.append({
                    'user_name': user_name,
                    'comment_text': comment_text,
                    'location': location
                })
            except Exception as e:
                print(f"Error parsing comment for code {code}: {e}")
                continue
        
        # 添加评论数据到 item
        item['comments'] = comments_data

        # 保存数据
        with open('_items_detail.txt', 'a+', encoding='utf-8') as f:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
        itemCodeList_detail.append(code)

        qbar.update(1)
    except Exception as e:
        print(f"Failed to parse item {code}: {e}")
        continue

    time.sleep(1)

# 完成
qbar.close()
print("Crawling completed!")

  0%|          | 0/102 [00:00<?, ?it/s]

Failed to load page for code 64c527d3000000001700ed8a: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF796CC6CB5+28821]
	(No symbol) [0x00007FF796C33840]
	(No symbol) [0x00007FF796AD578A]
	(No symbol) [0x00007FF796B291BE]
	(No symbol) [0x00007FF796B294AC]
	(No symbol) [0x00007FF796B72647]
	(No symbol) [0x00007FF796B4F33F]
	(No symbol) [0x00007FF796B6F412]
	(No symbol) [0x00007FF796B4F0A3]
	(No symbol) [0x00007FF796B1A778]
	(No symbol) [0x00007FF796B1B8E1]
	GetHandleVerifier [0x00007FF796FFFCAD+3408013]
	GetHandleVerifier [0x00007FF79701741F+3504127]
	GetHandleVerifier [0x00007FF79700B5FD+3455453]
	GetHandleVerifier [0x00007FF796D8BDBB+835995]
	(No symbol) [0x00007FF796C3EB5F]
	(No symbol) [0x00007FF796C3A814]
	(No symbol) [0x00007FF796C3A9AD]
	(No symbol) [0x00007FF796C2A199]
	BaseThreadInitThunk [0x00007FFB9355259D+29]
	RtlUserThreadStart [0x00007FFB9444AF38+40]

Failed to load page for code 649c1e74000000001300e49b: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF796CC6CB5+2882

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.85)
Stacktrace:
	GetHandleVerifier [0x00007FF796CC6CB5+28821]
	(No symbol) [0x00007FF796C33840]
	(No symbol) [0x00007FF796AD578A]
	(No symbol) [0x00007FF796AAF4F5]
	(No symbol) [0x00007FF796B56247]
	(No symbol) [0x00007FF796B6ECE2]
	(No symbol) [0x00007FF796B4F0A3]
	(No symbol) [0x00007FF796B1A778]
	(No symbol) [0x00007FF796B1B8E1]
	GetHandleVerifier [0x00007FF796FFFCAD+3408013]
	GetHandleVerifier [0x00007FF79701741F+3504127]
	GetHandleVerifier [0x00007FF79700B5FD+3455453]
	GetHandleVerifier [0x00007FF796D8BDBB+835995]
	(No symbol) [0x00007FF796C3EB5F]
	(No symbol) [0x00007FF796C3A814]
	(No symbol) [0x00007FF796C3A9AD]
	(No symbol) [0x00007FF796C2A199]
	BaseThreadInitThunk [0x00007FFB9355259D+29]
	RtlUserThreadStart [0x00007FFB9444AF38+40]


## 四、数据分析

In [ ]:
# 已经存储的详情页
if os.path.exists('_items_detail.txt'):
    with open('_items_detail.txt', 'r', encoding='utf-8') as f:
        items_detail = f.readlines()
    items_detail = [json.loads(item.strip()) for item in items_detail]
    
import pandas as pd
df_detail = pd.DataFrame(items_detail)

keywordsList = df_detail['keywords'].to_list()
keywordsList = [i.split(',') for i in keywordsList]



# 展平列表，对关键词进行计数
keywordsALL = [item for sublist in keywordsList for item in sublist]



# 看每一个关键词的数量
counts = {}
for i in keywordsALL:
    if i in counts.keys():
        counts[i] += 1
    else:
        counts[i] = 1



df = pd.DataFrame(counts.items(), columns=['keywords', 'counts'])
df.sort_values(by='counts', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(20)


完整代码（不包括detial部分）

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm import tqdm
from scrapy.selector import Selector
import json
import os
import time
import random

# 用户输入
text = input("请输入要搜索的关键词: ")
totalnum = int(input("请输入要爬取的帖子数量: "))

# 文件名设置
output_file = f"{text}_{totalnum}_items.txt"

# Selenium 配置
options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
options.add_argument('--incognito')  # 无痕模式
options.add_argument('blink-settings=imagesEnabled=false')  # 不加载图片
service = Service("C:/Users/lenovo/Desktop/xiaohongshu/chromedriver.exe")  # 替换为实际路径
browser = webdriver.Chrome(service=service, options=options)

# 打开小红书搜索页面
browser.get('https://www.xiaohongshu.com/explore')

# 判断是否登录
while True:
    page_source = browser.page_source
    if '登录探索更多内容' in page_source:
        print('%s : 未登录，请手动登录小红书！！！' % time.ctime())
        time.sleep(2)
    else:
        print('%s : 登录成功！！！' % time.ctime())
        time.sleep(3)
        break

# 打开搜索结果页面
url = f'https://www.xiaohongshu.com/search_result?keyword={text}&source=web_explore_feed'
browser.get(url)
time.sleep(3)

# 加载已有数据
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        items = [json.loads(item.strip()) for item in f.readlines() if item.strip()]
    itemCodeList = [item['code'] for item in items]
else:
    items = []
    itemCodeList = []

# 设置进度条
remaining_items = totalnum - len(itemCodeList)
qbar = tqdm(total=remaining_items)

# 解析网页内容
def parsePage(text):
    global items  # 确保更新到全局变量
    selector = Selector(text=text)
    divs = selector.xpath('//section[contains(@class, "note-item")]')
    if not divs:
        print("No items found on page.")
        return

    for div in divs:
        if len(items) >= totalnum:  # 检查是否达到爬取目标数量
            break

        item = {}
        try:
            # 提取信息
            item['title'] = div.xpath('.//a[@class="title"]/span/text()').extract_first()
            item['user'] = div.xpath('.//a[@class="author"]//span[@class="name"]/text()').extract_first()
            like_num = div.xpath('.//span[@class="count"]/text()').extract_first()
            item['likeNum'] = like_num.strip() if like_num else "0"
            
            # 提取 href
            href = div.xpath('.//a[@class="cover mask"]/@href').extract_first()
            if not href:
                href = div.xpath('.//a/@href').extract_first()
            if not href:
                print("Error parsing item: Missing href")
                continue
            item['code'] = href.split('/')[-1]

            # 保存数据
            if item['code'] not in itemCodeList:
                itemCodeList.append(item['code'])
                items.append(item)
                with open(output_file, 'a+', encoding='utf-8') as f:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')
                qbar.update(1)
        except Exception as e:
            print(f"Error parsing item: {e}")

# 爬取数据
while len(items) < totalnum:
    if '- THE END -' in browser.page_source:
        print('- THE END -')
        break

    parsePage(text=browser.page_source)
    time.sleep(random.uniform(1, 3))  # 添加随机延迟
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(random.uniform(2, 4))

qbar.close()
print(f"Crawling completed! Total items saved: {len(items)}")


Sun Nov 24 12:06:35 2024 : 登录成功！！！


  0%|          | 0/20 [00:00<?, ?it/s]

Error parsing item: Missing href
Error parsing item: Missing href


 85%|████████▌ | 17/20 [00:05<00:00,  3.38it/s]

Error parsing item: Missing href


100%|██████████| 20/20 [00:10<00:00,  2.00it/s]

Crawling completed! Total items saved: 20


In [12]:
#加入反扒
# "C:\Users\lenovo\Desktop\chrome-win64\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\selenum\AutomationProfile"
import os
import json
import time
import random
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from scrapy.selector import Selector

# 用户输入
text = input("请输入要搜索的关键词: ")
totalnum = int(input("请输入要爬取的帖子数量: "))

# 文件名设置
output_file = f"{text}_{totalnum}_items.txt"
output_detail_file = f"{text}_{totalnum}_items_detail.txt"

# 初始化 Selenium
def init_browser():
    options = Options()
    options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
    options.add_argument('--incognito')  # 无痕模式
    options.add_argument('--disable-blink-features=AutomationControlled')  # 反自动化检测
    options.add_argument('blink-settings=imagesEnabled=false')  # 不加载图片
    options.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36')
    service = Service("C:/Users/lenovo/Desktop/xiaohongshu/chromedriver.exe")  # 替换为实际路径
    browser = webdriver.Chrome(service=service, options=options)
    
    # 注入反检测脚本
    stealth_js = """
    Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
    window.navigator.chrome = { runtime: {} };
    Object.defineProperty(navigator, 'plugins', {get: () => [1, 2, 3, 4, 5]});
    Object.defineProperty(navigator, 'languages', {get: () => ['en-US', 'en']});
    """
    browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": stealth_js})
    return browser

browser = init_browser()

# 打开小红书搜索页面
browser.get('https://www.xiaohongshu.com/explore')

# 判断是否登录
while True:
    page_source = browser.page_source
    if '登录探索更多内容' in page_source:
        print('%s : 未登录，请手动登录小红书！！！' % time.ctime())
        time.sleep(2)
    else:
        print('%s : 登录成功！！！' % time.ctime())
        time.sleep(3)
        break

# 打开搜索结果页面
url = f'https://www.xiaohongshu.com/search_result?keyword={text}&source=web_explore_feed'
browser.get(url)
time.sleep(3)

# 加载已有数据
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        items = [json.loads(item.strip()) for item in f.readlines() if item.strip()]
    itemCodeList = [item['code'] for item in items]
else:
    items = []
    itemCodeList = []

if os.path.exists(output_detail_file):
    with open(output_detail_file, 'r', encoding='utf-8') as f:
        items_detail = [json.loads(item.strip()) for item in f.readlines() if item.strip()]
    itemCodeList_detail = [item['code'] for item in items_detail]
else:
    items_detail = []
    itemCodeList_detail = []

# 设置进度条
remaining_items = totalnum - len(itemCodeList)
qbar = tqdm(total=remaining_items)

# 解析网页内容
def parsePage(text):
    global items  # 确保更新到全局变量
    selector = Selector(text=text)
    divs = selector.xpath('//section[contains(@class, "note-item")]')
    if not divs:
        print("No items found on page.")
        return

    for div in divs:
        if len(items) >= totalnum:  # 检查是否达到爬取目标数量
            break

        item = {}
        try:
            # 提取信息
            item['title'] = div.xpath('.//a[@class="title"]/span/text()').extract_first()
            item['user'] = div.xpath('.//a[@class="author"]//span[@class="name"]/text()').extract_first()
            like_num = div.xpath('.//span[@class="count"]/text()').extract_first()
            item['likeNum'] = like_num.strip() if like_num else "0"
            
            # 提取 href
            href = div.xpath('.//a[@class="cover mask"]/@href').extract_first()
            if not href:
                href = div.xpath('.//a/@href').extract_first()
            if not href:
                print(f"Error parsing item: Missing href in div: {div.extract()}")
                continue
            item['code'] = href.split('/')[-1]

            # 保存数据
            if item['code'] not in itemCodeList:
                itemCodeList.append(item['code'])
                items.append(item)
                with open(output_file, 'a+', encoding='utf-8') as f:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')
                qbar.update(1)
        except Exception as e:
            print(f"Error parsing item: {e}")

# 爬取数据
while len(items) < totalnum:
    if '- THE END -' in browser.page_source:
        print('- THE END -')
        break

    parsePage(text=browser.page_source)
    time.sleep(random.uniform(1, 3))
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(random.uniform(2, 4))

qbar.close()
print(f"Crawling completed! Total items saved: {len(items)}")



Sun Nov 24 12:05:18 2024 : 登录成功！！！


  0%|          | 0/102 [04:13<?, ?it/s]


Error parsing item: Missing href in div: <section data-v-14f91c23="" data-v-7aab4d45="" class="note-item" data-index="5" style="--53f1d28e: 252.66666666666666px; --2db87601: 16px; --1013328c: blur(42.5px); transform: translate(276.667px, 440px);"><div data-v-14f91c23=""><div data-v-f2b40456="" data-v-14f91c23="" class="query-note-wrapper" style="--2054e94c: rgba(51, 51, 51, 0.80); --c165635e: rgba(222, 229, 240, 1); --b3e6f99e: rgba(51, 51, 51, 1); --c90bc876: rgba(235, 242, 250, 1); --43b33e67: rgba(66, 97, 125, 1);"><div data-v-f2b40456="" class="query-note-header"><div data-v-f2b40456="" class="icon-warpper"><svg data-v-23d27ada="" data-v-f2b40456="" class="reds-icon" width="20" height="20"><use data-v-23d27ada="" xlink:href="#search"></use></svg></div><span data-v-f2b40456="" class="query-note-header-text">相关搜索</span><!----></div><div data-v-f2b40456="" class="query-note-list"><div data-v-f2b40456="" class="query-note-item"><div data-v-f2b40456="" class="item-wrapper rec-query"><!-

Error parsing item: Missing href in div: <section data-v-14f91c23="" data-v-7aab4d45="" class="note-item" data-index="15" style="--53f1d28e: 252.66666666666666px; --2db87601: 16px; --1013328c: blur(42.5px); transform: translate(553.333px, 1948px);"><div data-v-14f91c23=""><div data-v-f2b40456="" data-v-14f91c23="" class="query-note-wrapper" style="--2054e94c: rgba(51, 51, 51, 0.80); --c165635e: rgba(222, 229, 240, 1); --b3e6f99e: rgba(51, 51, 51, 1); --c90bc876: rgba(235, 242, 250, 1); --43b33e67: rgba(66, 97, 125, 1);"><div data-v-f2b40456="" class="query-note-header"><div data-v-f2b40456="" class="icon-warpper"><svg data-v-23d27ada="" data-v-f2b40456="" class="reds-icon" width="20" height="20"><use data-v-23d27ada="" xlink:href="#search"></use></svg></div><span data-v-f2b40456="" class="query-note-header-text">相关搜索</span><!----></div><div data-v-f2b40456="" class="query-note-list"><div data-v-f2b40456="" class="query-note-item"><div data-v-f2b40456="" class="item-wrapper rec-query"><

100%|██████████| 20/20 [00:10<00:00,  1.83it/s]

Crawling completed! Total items saved: 20


In [4]:
##运行_items获得_items_detailed
import os
import json
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from scrapy.selector import Selector

# 文件名设置
output_items_file = "_items.txt"
output_details_file = "_items_detail.txt"

# 初始化 Selenium
def init_browser():
    options = Options()
    options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
    options.add_argument('--disable-blink-features=AutomationControlled')  # 防止被检测
    options.add_argument('--incognito')  # 无痕模式
    service_path = "C:/Users/lenovo/Desktop/xiaohongshu/chromedriver.exe"  # 替换为实际路径
    service = webdriver.chrome.service.Service(service_path)
    browser = webdriver.Chrome(service=service, options=options)
    return browser

browser = init_browser()

# 加载已有的 items 和详情数据
def load_existing_data():
    if os.path.exists(output_items_file):
        with open(output_items_file, 'r', encoding='utf-8') as f:
            items = [json.loads(line.strip()) for line in f if line.strip()]
    else:
        items = []

    if os.path.exists(output_details_file):
        with open(output_details_file, 'r', encoding='utf-8') as f:
            items_detail = [json.loads(line.strip()) for line in f if line.strip()]
    else:
        items_detail = []

    itemCodeList = [item['code'] for item in items]
    itemCodeList_detail = [item['code'] for item in items_detail]

    return items, items_detail, itemCodeList, itemCodeList_detail

items, items_detail, itemCodeList, itemCodeList_detail = load_existing_data()

# 爬取单个详情页数据
def parse_detail(code):
    test_url = f'https://www.xiaohongshu.com/discovery/item/{code}'
    try:
        browser.get(test_url)
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@class="list-container"]'))
        )
        html = browser.page_source
        selector = Selector(text=html)

        item_detail = {
            'code': code,
            'title': selector.xpath('//*[@name="og:title"]/@content').get(),
            'description': selector.xpath('//*[@name="description"]/@content').get(),
            'keywords': selector.xpath('//*[@name="keywords"]/@content').get(),
            'type': selector.xpath('//*[@name="og:type"]/@content').get(),
            'image': selector.xpath('//*[@name="og:image"]/@content').get(),
            'url': selector.xpath('//*[@name="og:url"]/@content').get(),
            'comments': []
        }

        # 提取评论信息
        comments = selector.xpath('//div[@class="parent-comment"]')
        for comment in comments:
            try:
                user_name = comment.xpath('.//div[@class="author"]/a[@class="name"]/text()').get()
                comment_text = ''.join(comment.xpath('.//div[@class="content"]//span[@class="note-text"]/span/text()').getall()).strip()
                location = comment.xpath('.//div[@class="info"]//span[@class="location"]/text()').get()
                item_detail['comments'].append({
                    'user_name': user_name,
                    'comment_text': comment_text,
                    'location': location
                })
            except Exception as e:
                print(f"Error parsing comment for code {code}: {e}")
                continue

        # 保存详情数据
        with open(output_details_file, 'a+', encoding='utf-8') as f:
            f.write(json.dumps(item_detail, ensure_ascii=False) + '\n')
        itemCodeList_detail.append(code)

        print(f"Saved details for code {code}, with {len(item_detail['comments'])} comments.")
    except Exception as e:
        print(f"Failed to load page for code {code}: {e}")

# 主程序：爬取 items 的详情数据
def crawl_details():
    global items
    remaining_items = [item for item in items if item['code'] not in itemCodeList_detail]
    qbar = tqdm(total=len(remaining_items))

    for item in remaining_items:
        parse_detail(item['code'])
        qbar.update(1)
        time.sleep(2)  # 避免过快请求被封

    qbar.close()
    print("Details crawling completed!")

# 输入要爬取的网址或直接运行
if __name__ == "__main__":
    # 爬取详情页数据
    crawl_details()


Failed to load page for code 64c527d3000000001700ed8a: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF796CC6CB5+28821]
	(No symbol) [0x00007FF796C33840]
	(No symbol) [0x00007FF796AD578A]
	(No symbol) [0x00007FF796B291BE]
	(No symbol) [0x00007FF796B294AC]
	(No symbol) [0x00007FF796B72647]
	(No symbol) [0x00007FF796B4F33F]
	(No symbol) [0x00007FF796B6F412]
	(No symbol) [0x00007FF796B4F0A3]
	(No symbol) [0x00007FF796B1A778]
	(No symbol) [0x00007FF796B1B8E1]
	GetHandleVerifier [0x00007FF796FFFCAD+3408013]
	GetHandleVerifier [0x00007FF79701741F+3504127]
	GetHandleVerifier [0x00007FF79700B5FD+3455453]
	GetHandleVerifier [0x00007FF796D8BDBB+835995]
	(No symbol) [0x00007FF796C3EB5F]
	(No symbol) [0x00007FF796C3A814]
	(No symbol) [0x00007FF796C3A9AD]
	(No symbol) [0x00007FF796C2A199]
	BaseThreadInitThunk [0x00007FFB9355259D+29]
	RtlUserThreadStart [0x00007FFB9444AF38+40]



Failed to load page for code 649c1e74000000001300e49b: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF796CC6CB5+28821]
	(No symbol) [0x00007FF796C33840]
	(No symbol) [0x00007FF796AD578A]
	(No symbol) [0x00007FF796B291BE]
	(No symbol) [0x00007FF796B294AC]
	(No symbol) [0x00007FF796B72647]
	(No symbol) [0x00007FF796B4F33F]
	(No symbol) [0x00007FF796B6F412]
	(No symbol) [0x00007FF796B4F0A3]
	(No symbol) [0x00007FF796B1A778]
	(No symbol) [0x00007FF796B1B8E1]
	GetHandleVerifier [0x00007FF796FFFCAD+3408013]
	GetHandleVerifier [0x00007FF79701741F+3504127]
	GetHandleVerifier [0x00007FF79700B5FD+3455453]
	GetHandleVerifier [0x00007FF796D8BDBB+835995]
	(No symbol) [0x00007FF796C3EB5F]
	(No symbol) [0x00007FF796C3A814]
	(No symbol) [0x00007FF796C3A9AD]
	(No symbol) [0x00007FF796C2A199]
	BaseThreadInitThunk [0x00007FFB9355259D+29]
	RtlUserThreadStart [0x00007FFB9444AF38+40]



Failed to load page for code 647fff9d0000000013010294: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.85)
Stacktrace:
	GetHandleVerifier [0x00007FF796CC6CB5+28821]
	(No symbol) [0x00007FF796C33840]
	(No symbol) [0x00007FF796AD578A]
	(No symbol) [0x00007FF796AAF4F5]
	(No symbol) [0x00007FF796B56247]
	(No symbol) [0x00007FF796B6ECE2]
	(No symbol) [0x00007FF796B4F0A3]
	(No symbol) [0x00007FF796B1A778]
	(No symbol) [0x00007FF796B1B8E1]
	GetHandleVerifier [0x00007FF796FFFCAD+3408013]
	GetHandleVerifier [0x00007FF79701741F+3504127]
	GetHandleVerifier [0x00007FF79700B5FD+3455453]
	GetHandleVerifier [0x00007FF796D8BDBB+835995]
	(No symbol) [0x00007FF796C3EB5F]
	(No symbol) [0x00007FF796C3A814]
	(No symbol) [0x00007FF796C3A9AD]
	(No symbol) [0x00007FF796C2A199]
	BaseThreadInitThunk [0x00007FFB9355259D+29]
	RtlUserThreadStart [0x00007FFB9444AF38+40]



KeyboardInterrupt: 

In [3]:
##关键字不行，但是单个网站可以（这是整合部分）
import os
import json
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from scrapy.selector import Selector

# 初始化 Selenium
def init_browser():
    options = Options()
    options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
    options.add_argument('--disable-blink-features=AutomationControlled')  # 防止被检测
    options.add_argument('--incognito')  # 无痕模式
    service_path = "C:/Users/lenovo/Desktop/xiaohongshu/chromedriver.exe"  # 替换为实际路径
    service = webdriver.chrome.service.Service(service_path)
    browser = webdriver.Chrome(service=service, options=options)
    return browser

browser = init_browser()

# 加载已有数据
def load_existing_data(filename):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return [json.loads(line.strip()) for line in f if line.strip()]
    return []

# 爬取单个详情页数据
def parse_detail(url, max_comments):
    try:
        browser.get(url)
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@class="list-container"]'))
        )
        html = browser.page_source
        selector = Selector(text=html)

        item_detail = {
            'url': url,
            'title': selector.xpath('//*[@name="og:title"]/@content').get(),
            'description': selector.xpath('//*[@name="description"]/@content').get(),
            'keywords': selector.xpath('//*[@name="keywords"]/@content').get(),
            'type': selector.xpath('//*[@name="og:type"]/@content').get(),
            'image': selector.xpath('//*[@name="og:image"]/@content').get(),
            'comments': []
        }

        # 提取评论信息
        comments = selector.xpath('//div[@class="parent-comment"]')
        for i, comment in enumerate(comments):
            if i >= max_comments:  # 限制评论数量
                break
            try:
                user_name = comment.xpath('.//div[@class="author"]/a[@class="name"]/text()').get()
                comment_text = ''.join(comment.xpath('.//div[@class="content"]//span[@class="note-text"]/span/text()').getall()).strip()
                location = comment.xpath('.//div[@class="info"]//span[@class="location"]/text()').get()
                item_detail['comments'].append({
                    'user_name': user_name,
                    'comment_text': comment_text,
                    'location': location
                })
            except Exception as e:
                print(f"Error parsing comment for {url}: {e}")

        print(f"Saved details for URL {url}, with {len(item_detail['comments'])} comments.")
        return item_detail
    except Exception as e:
        print(f"Failed to load page for {url}: {e}")
        return None

# 主函数
def main():
    user_input = input("请输入关键字或网址: ").strip()
    max_comments = int(input("请输入每条需要的评论量: "))

    # 如果输入的是关键字
    if "http" not in user_input:
        items_file = f"{user_input}_items.txt"
        items = load_existing_data(items_file)
        if not items:
            print(f"未找到文件: {items_file}，请确认关键字是否正确。")
            return
        print(f"成功加载 {len(items)} 条帖子信息。")
        
        details_file = f"{user_input}_items_detail.txt"
        processed_urls = [item['url'] for item in load_existing_data(details_file)]
        qbar = tqdm(total=len(items))

        # 爬取详情数据
        for item in items:
            url = f"https://www.xiaohongshu.com/discovery/item/{item['code']}"
            if url in processed_urls:
                continue
            detail = parse_detail(url, max_comments)
            if detail:
                with open(details_file, 'a+', encoding='utf-8') as f:
                    f.write(json.dumps(detail, ensure_ascii=False) + '\n')
            time.sleep(2)  # 避免过快请求被封
            qbar.update(1)
        qbar.close()
        print("关键字爬取完成！")

    # 如果输入的是网址
    else:
        details_file = "_items_detail.txt"
        print(f"处理单个网址: {user_input}")
        detail = parse_detail(user_input, max_comments)
        if detail:
            with open(details_file, 'a+', encoding='utf-8') as f:
                f.write(json.dumps(detail, ensure_ascii=False) + '\n')
        print("单个网址爬取完成！")

if __name__ == "__main__":
    main()


成功加载 10 条帖子信息。


  0%|          | 0/10 [00:00<?, ?it/s]

Failed to load page for https://www.xiaohongshu.com/discovery/item/64561f370000000013007e99: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF796CC6CB5+28821]
	(No symbol) [0x00007FF796C33840]
	(No symbol) [0x00007FF796AD578A]
	(No symbol) [0x00007FF796B291BE]
	(No symbol) [0x00007FF796B294AC]
	(No symbol) [0x00007FF796B72647]
	(No symbol) [0x00007FF796B4F33F]
	(No symbol) [0x00007FF796B6F412]
	(No symbol) [0x00007FF796B4F0A3]
	(No symbol) [0x00007FF796B1A778]
	(No symbol) [0x00007FF796B1B8E1]
	GetHandleVerifier [0x00007FF796FFFCAD+3408013]
	GetHandleVerifier [0x00007FF79701741F+3504127]
	GetHandleVerifier [0x00007FF79700B5FD+3455453]
	GetHandleVerifier [0x00007FF796D8BDBB+835995]
	(No symbol) [0x00007FF796C3EB5F]
	(No symbol) [0x00007FF796C3A814]
	(No symbol) [0x00007FF796C3A9AD]
	(No symbol) [0x00007FF796C2A199]
	BaseThreadInitThunk [0x00007FFB9355259D+29]
	RtlUserThreadStart [0x00007FFB9444AF38+40]



 10%|█         | 1/10 [00:14<02:06, 14.05s/it]

Failed to load page for https://www.xiaohongshu.com/discovery/item/664d578c000000001501009e: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.85)
Stacktrace:
	GetHandleVerifier [0x00007FF796CC6CB5+28821]
	(No symbol) [0x00007FF796C33840]
	(No symbol) [0x00007FF796AD578A]
	(No symbol) [0x00007FF796AAF4F5]
	(No symbol) [0x00007FF796B56247]
	(No symbol) [0x00007FF796B6ECE2]
	(No symbol) [0x00007FF796B4F0A3]
	(No symbol) [0x00007FF796B1A778]
	(No symbol) [0x00007FF796B1B8E1]
	GetHandleVerifier [0x00007FF796FFFCAD+3408013]
	GetHandleVerifier [0x00007FF79701741F+3504127]
	GetHandleVerifier [0x00007FF79700B5FD+3455453]
	GetHandleVerifier [0x00007FF796D8BDBB+835995]
	(No symbol) [0x00007FF796C3EB5F]
	(No symbol) [0x00007FF796C3A814]
	(No symbol) [0x00007FF796C3A9AD]
	(No symbol) [0x00007FF796C2A199]
	BaseThreadInitThunk [0x00007FFB9355259D+29]
	RtlUserThreadStart [0x00007FFB9444AF38+40]



KeyboardInterrupt: 